In [2]:
import importlib
import sys
from tqdm import tqdm
import  os
import pickle
import glob
from datetime import datetime
import matplotlib.pyplot as plt
# Add the directories to the sys.path
dir_path = "C://Users//jony//Programming//Python//Anubis//anubis//" # insert your directory path
sys.path.append(dir_path + "Osiris//processing//python")
sys.path.append(dir_path + "tools")

import Analysis_tools as ATools
import Atlas_tools as AtTools
import proAnubis_Analysis_Tools
import Reconstruction_tools as RTools
import numpy as np
import mplhep as hep
import Timing_tools as TTools
import gc
import pickle
import overview
import rawFileReader
import Visual_tools as VTools
hep.style.use([hep.style.ATLAS])

dir = "C://Users//jony//Programming//Python//Anubis//anubis//data//"

# Specify the directory


In [3]:
importlib.reload(AtTools)
analyser = AtTools.AtlasAnalyser()
start = "2024-09-05 06:43:00" 
end = "2024-09-05 07:10:23"
file_name = "proAnubis_240905_0635.raw"

analyser.get_tdc5_data(file_name, 1, 2, 100)
initial_time = analyser.anubis_data[0].event_time
print(initial_time)
print(datetime.timestamp(initial_time))
event_time = initial_time

start = datetime.strptime(start, '%Y-%m-%d %H:%M:%S')
end = datetime.strptime(end, '%Y-%m-%d %H:%M:%S')
previous_event_time = initial_time
with tqdm(total=round((start-initial_time).total_seconds()), desc=f"Skipping {file_name}", unit='tdc5 Reads') as pbar:
    while event_time < start:
        analyser.fReader.skip_events(20_000)
        event_time = None
        while not event_time:
            if not analyser.fReader.readBlock():
                    raise EOFError("You have reached the end of the file")
            if analyser.fReader.hasEvents():
                tdc5Reads = analyser.fReader.getTDCFiveEvents()
                if not tdc5Reads:
                    continue
                else:
                    event_time = tdc5Reads[0][0]
                    pbar.update(event_time.timestamp() - previous_event_time.timestamp())
                    previous_event_time = event_time

previous_event_time = start
print(event_time)
with tqdm(total=round((end-start).total_seconds()), desc=f"Reading {file_name}", unit='seconds') as pbar:        
    while event_time < end:
        gc.disable()
        analyser.anubis_data = []
        try:
            analyser.get_tdc5_data(file_name, 1, 2, amount_of_events=20_000)
        except EOFError:
            file_name = "proAnubis_240905_0635.raw"
    
        event_time = analyser.anubis_data[-1].event_time
        print("Event time:", event_time)
        print("Timestamp:", analyser.anubis_data[-1].timeStamp)
        pbar.update(round(event_time.timestamp()- previous_event_time.timestamp()))
        previous_event_time = event_time
        with open(dir + f"correct_bcr//{round(event_time.timestamp())}", "wb") as f:
            pickle.dump(analyser.anubis_data, f)
        gc.enable()
        gc.collect()
        

            

New
2024-09-05 05:35:00.028855
1725510900.028855


Skipping proAnubis_240905_0635.raw: 3441.4411067962646tdc5 Reads [04:38, 12.34tdc5 Reads/s]                  


2024-09-05 06:32:21.469962


Reading proAnubis_240905_0635.raw:   5%|▍         | 51/1083 [00:13<04:29,  3.83seconds/s]

Event time: 2024-09-05 06:33:11.337475
Timestamp: 1725514391.337475


...

Event time: 2024-09-05 06:41:59.976777
Timestamp: 1725514919.976777


Reading proAnubis_240905_0635.raw:  58%|█████▊    | 632/1083 [07:57<08:51,  1.18s/seconds]

Event time: 2024-09-05 06:42:52.380086
Timestamp: 1725514972.380086


Reading proAnubis_240905_0635.raw:  58%|█████▊    | 632/1083 [17:03<12:10,  1.62s/seconds]


KeyboardInterrupt: 

In [42]:
importlib.reload(AtTools)

analyser = AtTools.AtlasAnalyser() # 784_480
data_list = sorted([f for f in os.listdir("data//correct_bcr") if os.path.isfile(os.path.join("data//correct_bcr", f))], key=lambda x: int(x[3:-4]))
rates = []
times = []
with tqdm(total=len(data_list)) as pbar:
    for data in data_list:
        gc.disable()
        with open(f"data//correct_bcr//{data}", "rb") as f:
            analyser.anubis_data += pickle.load(f)
        analyser.get

        pbar.update(1)
        gc.enable()
        gc.collect()
print(len(rates)) 


 76%|███████▌  | 25/33 [01:08<00:21,  2.73s/it]


KeyboardInterrupt: 

In [23]:
importlib.reload(AtTools)
atlas_file = "C://Users//jony//Programming//Python//Anubis//anubis//data//dataAtlas2.root"
analyser = AtTools.AtlasAnalyser() # 784_480
data_list = sorted([f for f in os.listdir("data//correct_bcr") if os.path.isfile(os.path.join("data//correct_bcr", f))], key=lambda x: int(x[3:-4]))
total_matches = []
times = []
trigger_rates = []
with tqdm(total=len(data_list)) as pbar:
    for i, data in enumerate(data_list):
        gc.disable()
        with open(f"data//correct_bcr//{data}", "rb") as f:
            analyser.anubis_data += pickle.load(f)
        analyser.get_atlas_data(atlas_file)
        analyser.atlas_data = analyser.atlas_data[(analyser.anubis_data[0].timeStamp < analyser.atlas_data["TimeStamp"]) & (analyser.atlas_data["TimeStamp"] < analyser.anubis_data[-1].timeStamp)]
        analyser.match_bcrs()
        total_matches.extend(analyser.matches)
        tr, ti = analyser.trigger_rates()
        trigger_rates.append(tr)
        times.append(ti)
        analyser.anubis_data = []
        pbar.update(1)
        gc.enable()
        gc.collect()
        if i % 10 == 0:
            with open("data//total_matches_400", "wb") as f:
                pickle.dump(total_matches, f)



Matching: 0Events [00:00, ?Events/s]s]
Matching: 0Events [00:00, ?Events/s] 5.46s/it]
Matching: 0Events [00:00, ?Events/s] 6.91s/it]
Matching: 0Events [00:00, ?Events/s] 6.16s/it]
Matching: 0Events [00:00, ?Events/s] 5.81s/it]
Matching: 100%|██████████| 96/96 [00:02<00:00, 43.97Events/s]
Matching: 0Events [00:00, ?Events/s] 29.43s/it]
Matching: 0Events [00:00, ?Events/s] 22.34s/it]
Matching: 0Events [00:00, ?Events/s] 17.42s/it]
Matching: 0Events [00:00, ?Events/s] 13.93s/it]
Matching: 0Events [00:00, ?Events/s] 11.48s/it]
100%|██████████| 72/72 [1:09:13<00:00, 57.68s/it] 


In [45]:
data = analyser.anubis_data.copy()

In [46]:
importlib.reload(AtTools)
analyser = AtTools.AtlasAnalyser()
analyser.anubis_data = data.copy()

In [ ]:
#generate random data
importlib.reload(ATools)
data = AtTools.generate_random_triggers(1_000_000)
hist, bins = np.histogram(data, bins=list(range(0, 3654)), density=True)
plt.plot(bins[:-1], hist)
plt.title("Random triggers")
plt.xlabel("Bunch Crossings [bx]")
plt.ylabel("Frequency")
plt.show()

In [3]:
#Atlas load
importlib.reload(AtTools)
atlas_file = "C://Users//jony//Programming//Python//Anubis//anubis//data//dataAtlas2.root"
analyser.get_atlas_data(atlas_file)
print("Loading done")

Loading done


In [20]:
#Basic analysis
print("Anubis len:", len(analyser.anubis_data))
print("ATLAS len:", len(analyser.atlas_data))
print("Atlas first:",analyser.atlas_data.iloc[0]["TimeStamp"])
print("Anubis first:",analyser.anubis_data[0].timeStamp)
print("Atlas last:", analyser.atlas_data.iloc[-1]["TimeStamp"])
print("Anubis last:", analyser.anubis_data[-1].timeStamp)

Anubis len: 0
ATLAS len: 942478
Atlas first: 1725509842


IndexError: list index out of range

In [9]:
#Trigger rates
trigger_rates, times = analyser.trigger_rates()
hist, bins = analyser.beam_luminosity()
hist = max(trigger_rates)/max(hist)*hist

plt.plot(bins[:-1], hist, label="Luminosity ATLAS")
plt.step(times, trigger_rates, where="post", label="Trigger rates")
plt.legend()
plt.xlabel("TimeStamp")
#plt.xlim([times[0], times[-1]])
plt.show()


In [8]:
#Same times
analyser.atlas_data = analyser.atlas_data[(analyser.anubis_data[0].timeStamp < analyser.atlas_data["TimeStamp"]) & (analyser.atlas_data["TimeStamp"] < analyser.anubis_data[-1].timeStamp)]

In [13]:
#Check offset
AtTools.plot_offset(analyser)

Best offset 114


In [7]:
fake_trigger_rates = []
for tr in trigger_rates:
    fake_trigger_rates.extend(tr)

fake_times = []
for ti in times:
    fake_times.extend(ti)

atlas_file = "C://Users//jony//Programming//Python//Anubis//anubis//data//dataAtlas2.root"
analyser.get_atlas_data(atlas_file)
hist, bins = analyser.beam_luminosity()
plt.plot(fake_times, fake_trigger_rates, label="Trigger rates")
plt.plot(bins[:-1], hist, label="Luminosity")
plt.show()

In [24]:
analyser.matches = total_matches

In [10]:
analyser.match_bcrs() #34705

print("Number of matches:", len([1 for i in analyser.matches if i[1]]))

Number of matches: 3105


In [11]:
print("Number of matches:", len([1 for i in analyser.matches if i[1]]))

Number of matches: 3105


In [25]:
importlib.reload(AtTools)
times, eta, phi, heatmap = AtTools.convert_matches(analyser.matches, 114)

In [29]:
import background

bg = background.calculate_background(anubis_data=analyser.anubis_data)


100%|██████████| 3564/3564 [00:02<00:00, 1507.49it/s]


In [30]:
#Position

plt.hist2d(eta, phi, bins=30, cmap='plasma')
plt.xlim(-2.5, 2.5)
plt.ylim(-np.pi, np.pi)
plt.xlabel("eta")
plt.ylabel("phi")
plt.colorbar()
plt.show()

In [29]:
# position 3d
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
hist, xedges, yedges = np.histogram2d(eta, phi, bins=30, range=[[-np.pi, np.pi], [-np.pi, np.pi]])

xpos, ypos = np.meshgrid(xedges[:-1] + 0.5 * (xedges[1] - xedges[0]),
                         yedges[:-1] + 0.5 * (yedges[1] - yedges[0]),
                         indexing="ij")

# Initialize the 3D plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Plot the surface
ax.plot_surface(xpos, ypos, hist, cmap='plasma')

# Set labels
ax.set_xlabel('eta')
ax.set_ylabel('phi')
ax.set_zlabel('Count')

# Set limits
ax.set_xlim(-np.pi, np.pi)
ax.set_ylim(-np.pi, np.pi)
plt.show()


In [28]:
#times
plt.hist(times, bins=range(min(times), max(times) + 2, 1), density=True)
#plt.plot(range(0, 3564), bg)
#plt.xlim(-100,100)
plt.xlabel("Time difference (bx)")
#plt.xlim(-500,500)
plt.ylabel("Frequency")
plt.title("Time difference between BCR and trigger")
plt.show()

In [21]:
#compute correlation from heatmap
atlas = []#AtTools.generate_random_triggers(10_000_000)
anubis = []#AtTools.generate_random_triggers(10_000_000)

for i in range(len(heatmap)):
    for j in range(len(heatmap[i])):
        for count in range(heatmap[i][j]):
            atlas.append(i)
            anubis.append(j)

from scipy.stats import pearsonr
r, p = pearsonr(atlas, anubis)
print("Pearson coefficient:", r)
print("p-value:", p)


Pearson coefficient: -0.003686873447099135
p-value: 0.8177745494027553


In [54]:
print(len(atlas))

204801


In [55]:
#histogram of the matched bcrs
anubis = []
for match in analyser.matches:
    if match[1] != []:
        anubis.extend(match[1])
hist = []
for trigger in anubis:
    hist.append(round(trigger.bcId))

bins = [i-0.5 for i in range(0,3565)] #-0.5,0.5,...3653.5
counts, _ = np.histogram(hist, bins=bins, density=False)
bins = [i for i in range(0,3564)]
plt.plot(bins, counts, color="orange")

plt.xlabel('Time since last BCR (ns)')
plt.xlim(0, 3565)
plt.ylabel('Frequency')
plt.title(f'Histogram of BCID Anubis')
plt.show()

In [43]:
#heatmap
plt.imshow(heatmap)
plt.plot([i for i in range(3564)],[i for i in range(3564)], color = "red")
plt.xlim(0, 3564)
plt.ylim(0, 3564)
#plt.gca().invert_yaxis()
plt.xlabel("Atlas BX")
plt.ylabel("proANUBIS BX")
plt.title("Heatmap of BX matches")
plt.show()